# Azure OpenAI -  Content generation
 
Given a prompt, the model will return one or more predicted completions, and can also return the probabilities of alternative tokens at each position.

source: https://learnprompting.org/docs/basic_applications/writing_emails

In [3]:
import json
import openai
import os

### Setup Parameters


Here we will load the configurations from _config.json_ file to setup deployment name, openai api base, openai api key and openai api version.

In [4]:
# Setting up the deployment name
deployment_name = "gpt-35turbo"

# This is set to `azure`
openai.api_type = "azure"
openai.api_version = "2023-03-15-preview"

# The API key for your Azure OpenAI resource.
API_KEY = "d1869cee351446e2bc6b6ffef2207576"
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = "https://azure-openai-test21.openai.azure.com/"
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
openai.api_base = RESOURCE_ENDPOINT

In [5]:
def run_completion(prompt: str, deployment_name: str, temperaure=0.7, max_tokens=100, verbose=False):
    try:
        # Create a completion for the provided prompt and parameters
        # To know more about the parameters, checkout this documentation: https://learn.microsoft.com/en-us/azure/cognitive-services/openai/reference
        completion = openai.Completion.create(
                        prompt=prompt,
                        temperature=temperaure,
                        max_tokens=max_tokens,
                        top_p=1,
                        frequency_penalty=0,
                        presence_penalty=0,
                        engine=deployment_name)

        # print the completion
        if (verbose):
            print(completion.choices[0].text.strip(" \n"))
        
        return completion.choices[0].text
        
        # Here indicating if the response is filtered
        if completion.choices[0].finish_reason == "content_filter":
            print("The generated content is filtered.")
            
    except openai.error.APIError as e:
        # Handle API error here, e.g. retry or log
        print(f"OpenAI API returned an API Error: {e}")

    except openai.error.AuthenticationError as e:
        # Handle Authentication error here, e.g. invalid API key
        print(f"OpenAI API returned an Authentication Error: {e}")

    except openai.error.APIConnectionError as e:
        # Handle connection error here
        print(f"Failed to connect to OpenAI API: {e}")

    except openai.error.InvalidRequestError as e:
        # Handle connection error here
        print(f"Invalid Request Error: {e}")

    except openai.error.RateLimitError as e:
        # Handle rate limit error
        print(f"OpenAI API request exceeded rate limit: {e}")

    except openai.error.ServiceUnavailableError as e:
        # Handle Service Unavailable error
        print(f"Service Unavailable: {e}")

    except openai.error.Timeout as e:
        # Handle request timeout
        print(f"Request timed out: {e}")

In [6]:
def print_response(r):
    for x in r.split("\n"):
        print(x)


## Responding to an Email

Two-way / two-hop approach:

1. Generate summary and action points
1. Generate reply based on the summary and action points

In [7]:
prompt = """
Dear Mario,

I hope you're doing well. I am writing to provide you with some updates on our company and to request your assistance in addressing an issue with the user interface (UI) of the software you have been working on.

As you know, our company has been focusing on providing a user-friendly experience to our customers, and we have identified some issues with the UI of the software. Specifically, we have received feedback from our users that they are having difficulty accessing certain features, and that the UI is not intuitive enough.

We need your help to address these issues and make improvements to the UI of the software. This may involve making changes to the layout, design, or functionality of the UI. We believe that your expertise in software development and your familiarity with the software will make you the ideal person to help us with this task.

If you need any additional resources or support to complete this task, please do not hesitate to let me know. Additionally, I would appreciate it if you could provide me with an estimated timeline for when you expect these modifications to be completed.

If you have any questions or need further clarification, please do not hesitate to contact me.

Thank you for your hard work and dedication to our company. I look forward to hearing back from you soon.

Best regards,
John

Generate a summary of this and a list of action items.
"""
r = run_completion(prompt=prompt, deployment_name=deployment_name, temperaure=1.0, max_tokens=500, verbose=False)
print_response(r)

"""

Summary:
The user interface (UI) of our software is not user-friendly, as confirmed by our customers' feedback. Therefore, we need to make improvements to the UI layout, design, or functionality with the help of Mario who is working on the software. This is essential to provide a better experience for our customers.
 
Action items:
1) Write an email to Mario requesting his help with the UI modifications.
2) Schedule a meeting with Mario to discuss the changes needed and provide additional resources or support if necessary.
3) Provide a timeline for when we expect these modifications to be completed.
4) Ensure customer feedback is collected and analyzed to determine whether additional changes are needed.   

"""

Dear Mario,

I hope you're doing well. We have identified some issues with the user interface (UI) of our software. Our clients are finding it hard to access some features, leading to complaints about the immutability of the UI. Therefore, we are seeking your help to impro

In [8]:
prompt = """
Summary: John is writing to inform Mario of recent feedback the company has received regarding their user interface (UI) of the software. John is requesting Mario's help in addressing these issues and making improvements to the software's UI. 

Action Items:
1. Assess the layout, design, and functionality of the UI and make necessary changes.
2. Provide John with an estimated timeline for when these modifications are expected to be complete.
3. Let John know if any additional resources or support are needed to complete the task.

Write a response email from Mario using the above email summary:
"""

r = run_completion(prompt=prompt, deployment_name=deployment_name, temperaure=1.0, max_tokens=500, verbose=False)
print_response(r)

Dear John,

Thank you for sending me the feedback report on our UI. I appreciate the insight you have provided and I'm willing to make the necessary changes to improve our UI. I will conduct a thorough assessment and see where improvements can be made.

I should be able to provide an estimated timeline for the completion of the necessary modifications by the end of the week. If there are any additional resources or support needed, do let me know and we can work together to get everything done timely.

Thank you again for bringing this to my attention.

Best regards,
Mario.
